In [ ]:
#To install wandb package ,which is helpful in generating plots and report.
!pip install wandb

In [ ]:
# Essentials
import numpy as np
import tensorflow
from tensorflow import keras
from keras import regularizers
from keras.models import Sequential
from keras.utils import np_utils
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Activation
import wandb
from wandb.keras import WandbCallback
from sklearn.model_selection import train_test_split

import random
import imageio
import os
import cv2
import glob
# random.seed(42)

In [ ]:
# Load Dataset
!git clone https://github.com/ashwanth10/INaturalist_Dataset.git

fatal: destination path 'INaturalist_Dataset' already exists and is not an empty directory.


In [ ]:
# Labels for INaturalist Dataset
class_labels = {
    0: 'Amphibia',
    1: 'Animalia',
    2: 'Arachnida',
    3: 'Aves',
    4: 'Fungi',
    5: 'Insecta', 
    6: 'Mammalia', 
    7: 'Mollusca', 
    8: 'Plantae',
    9: 'Reptilia'
}

num_classes = 10
img_size = 128
directory = 'INaturalist_Dataset/train'

# Load training data
x_train = []
y_train = []
for label, name in class_labels.items():
   list_images = os.listdir(directory + '/' + name)
   for image_name in list_images:
       try:
          image = imageio.imread(directory +'/'+name+'/'+image_name)
       except: 
          continue
       if np.ndim(image) == 3:
          x_train.append(cv2.resize(image, (img_size, img_size)))
          y_train.append(label)
       else: 
          # Some images are black and white
          print(image_name)

aa66b3ba5ccffdb3c41c1d3acf7dd569.jpg
2fad1ed3a05629c089cd530f297e27e6.jpg
d192b73b0af85824c65e91f8795d8df7.jpg


In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
# Set seed
np.random.seed(0)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
                                                  test_size=0.1, 
                                                  stratify=y_train,
                                                  random_state=42)

In [ ]:
# normalize data
x_train = x_train / 255
x_val = x_val / 255

In [ ]:
# One-hot encoding
targs = np.zeros((len(y_train), 10, 1))
for i in range(len(y_train)):
    targs[i][y_train[i]][0] = 1 # preparing the target matrix 

y_train = targs

In [ ]:
targs_val = np.zeros((len(y_val), 10, 1))
for i in range(len(y_val)):
    targs_val[i][y_val[i]][0] = 1 # preparing the target matrix 

y_val = targs_val

In [ ]:
sweep_config = {
    'method': 'random', 
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'kernel_size':{
            'values': [[(3,3),(3,3),(3,3),(3,3),(3,3)], [(3,3),(5,5),(5,5),(7,7),(7,7)], [(7,7),(7,7),(5,5),(5,5),(3,3)], [(3,3),(5,5),(7,7),(9,9),(11,11)] ]
        },
        'weight_decay': {
            'values': [0, 0.0005, 0.005]
        },
        'dropout': {
            'values': [0, 0.2, 0.4]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },
        'activation': {
            'values': ['relu', 'elu', 'selu']
        },
        'batch_norm':{
            'values': ['true','false']
        },
        'filt_org':{
            'values': [[32,32,32,32,32],[32,64,64,128,128],[128,128,64,64,32],[32,64,128,256,512]]
        },
        'data_augment': {
            'values': ['true','false']
        },
        'batch_size': {
            'values': [32, 64]
        },
        'num_dense':{
            'values': [64, 128, 256, 512]
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="cs21m010-cs21m041", project="DL_Assignment_2")

In [ ]:
def addActivation(model, config):
    if config.activation == "relu":
        model.add(Activation('relu'))
    elif config.activation == "elu":
        model.add(Activation('elu'))
    elif config.activation == "selu":
        model.add(Activation('selu'))

In [ ]:
def configureBatchNormalization(model, config):
    if config.batch_norm == 'True':
        model.add(BatchNormalization())

In [ ]:
def train():

    # Initialize a new wandb run
    wandb.init(config=sweep_config)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    wandb.run.name = 'num_dense_'+ str(config.num_dense)+'_bs_'+str(config.batch_size)+'_ac_'+ config.activation
    
    # Determine input shape
    input_shape = (img_size, img_size , 3)
    
    # Define the model architecture
    model = Sequential()

    filter = config.filt_org

    for i in range(5):
        model.add(Conv2D(filters = filter[i], kernel_size = config.kernel_size[i],
                         padding = 'same', input_shape = input_shape, 
                         kernel_regularizer=regularizers.l2(config.weight_decay)))
        
        addActivation(model, config)
        configureBatchNormalization(model, config)
        model.add(MaxPooling2D(pool_size=(2, 2)))  

    # FC layer
    model.add(Flatten())
    model.add(Dense(config.num_dense, activation = config.activation, kernel_regularizer = regularizers.l2(config.weight_decay)))
    model.add(Dropout(config.dropout))
    model.add(BatchNormalization())

    # Output layer
    model.add(Dense(num_classes, activation = "softmax"))

    # Define the optimizer
    optimizer = Adam(lr=config.learning_rate, beta_1=0.9, beta_2=0.999)
    
    model.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics=['accuracy'])

    #data augmentation
    if config.data_augment == 'true':
        datagen = ImageDataGenerator(
            rotation_range=45,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False  # randomly flip images
        )
    else:
        datagen = ImageDataGenerator(rescale = 1.0)

    datagen.fit(x_train)
    
    model.fit(
        datagen.flow(x_train, y_train, batch_size = config.batch_size),
        epochs = 10,
        verbose = 1,
        validation_data= (x_val, y_val),
        callbacks = [WandbCallback()]
    )
    

In [ ]:
wandb.agent(sweep_id, train, count = 1)